In [ ]:
import pyPhyNR as pynr


In [ ]:
# TDD Configuration
TDD_SLOT_PATTERN = [0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SPECIAL_SLOT_SYMBOL_PATTERN = [0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 1, 1, 1, 1]

print("TDD Slot Pattern (20 slots):")
print(f"Pattern: {TDD_SLOT_PATTERN}")
print(f"DL slots: {[i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 0]}")
print(f"UL slots: {[i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 1]}")
print(f"Special slots: {[i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 2]}")

print("\nSpecial Slot Symbol Pattern (14 symbols):")
print(f"Pattern: {SPECIAL_SLOT_SYMBOL_PATTERN}")
print(f"DL symbols: {[i for i, sym in enumerate(SPECIAL_SLOT_SYMBOL_PATTERN) if sym == 0]}")
print(f"UL symbols: {[i for i, sym in enumerate(SPECIAL_SLOT_SYMBOL_PATTERN) if sym == 1]}")
print(f"Guard period: {[i for i, sym in enumerate(SPECIAL_SLOT_SYMBOL_PATTERN) if sym == 2]}")
print(f"Switch point: {[i for i, sym in enumerate(SPECIAL_SLOT_SYMBOL_PATTERN) if sym == 3]}")


In [ ]:
# Create signal builder for 10MHz, numerology 1 (30kHz SCS)
signal = pynr.NRSignalBuilder(bandwidth_mhz=10, numerology=1, cell_id=1)
signal.configure_carrier(sample_rate=11.52e6, fft_size=384).initialize_grid()
print(signal.get_carrier_config())

In [ ]:
# Add PDSCH to DL slots
dl_slots = [i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 0]
signal.add_pdsch(
    start_rb=0,
    num_rb=24,
    start_symbol=0,
    num_symbols=14,
    slot_pattern=dl_slots,
    modulation="QAM256",
    power=0.0
).add_dmrs(
    dmrs_positions=[2, 11],
    clear_full_symbol=False,
    subcarrier_pattern="even",
    power_offset_db=0.0
)


In [ ]:
# Add PDSCH to special slot
special_slots = [i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 2]
dl_symbols_in_special = SPECIAL_SLOT_SYMBOL_PATTERN.count(0)

if special_slots:
    signal.add_pdsch(
        start_rb=0,
        num_rb=24,
        start_symbol=0,
        num_symbols=dl_symbols_in_special,
        slot_pattern=special_slots,
        modulation="QAM256",
        power=0.0
    ).add_dmrs(
        dmrs_positions=[2, 7],
        clear_full_symbol=False,
        subcarrier_pattern="even",
        power_offset_db=0.0
    )


In [ ]:
# Generate waveform
pynr.utils.plot_grid_dl(signal.carrier_config, signal.grid)
iq_samples = signal.generate_signal(target_rms=2914)
pynr.utils.plot_time_domain(iq_samples, signal.carrier_config)
pynr.utils.plot_frequency_domain(iq_samples, signal.carrier_config)
